# Imports

## Application-specific imports 

In [1]:
import sys

In [2]:
sys.path.append("../config/")
import config

In [3]:
sys.path.append("../metaflow/")
import preprocess_fn

## General 

In [1]:
import json
import numpy as np

In [3]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 113 kB/s  eta 0:00:01    |██████████▍                     | 91.1 MB 5.5 MB/s eta 0:00:35
     |████████████████████████████████| 199 kB 6.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764027 sha256=b61615f29e7dfb46680cb75302a07f883e72dc99c5f71600cbf27e3fc44abf91
  Stored in directory: /home/makism/.cache/pip/wheels/05/75/73/81f84d174299abca38dd6a06a5b98b08ae25fce50ab8986fa1
Successfully built pyspark


In [2]:
from pyspark.sql import functions as fn

ModuleNotFoundError: No module named 'pyspark'

In [6]:
spark = preprocess_fn.spark_session()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/09 16:34:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load the set from JSON

In [7]:
code = "M20"

In [8]:
df = spark.read.json('../artifacts/dataset/M20.json')

In [9]:
df.count()

345

In [10]:
df.where(df.name == "Bonecrusher Giant").toPandas()

22/04/09 16:35:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,artist,borderColor,colorIdentity,colors,convertedManaCost,edhrecRank,flavorText,foreignData,frameEffect,frameEffects,...,scryfallOracleId,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations


In [11]:
df.where(df.name == "Stomp").toPandas()

,artist,borderColor,colorIdentity,colors,convertedManaCost,edhrecRank,flavorText,foreignData,frameEffect,frameEffects,...,scryfallOracleId,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations


# Preprocess 

## Filter out duplicate cards

In [12]:
df_filtered = preprocess_fn.remove_duplicate_cards(df)

In [13]:
num_cards = df_filtered.count()
print(f"Final number of cards {num_cards}")

Final number of cards 329


In [14]:
df_filtered.where(df_filtered.name == "Bonecrusher Giant").toPandas()

,number,artist,borderColor,colorIdentity,colors,convertedManaCost,edhrecRank,flavorText,foreignData,frameEffect,...,scryfallOracleId,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations


In [15]:
df_filtered.where(df_filtered.name == "Stomp").toPandas()

,number,artist,borderColor,colorIdentity,colors,convertedManaCost,edhrecRank,flavorText,foreignData,frameEffect,...,scryfallOracleId,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations


## Drop columns 

In [16]:
df_filtered = preprocess_fn.drop_columns(df_filtered)

## Filter the text

In [17]:
df_filtered = df_filtered.withColumn('filtered_text',
                                     preprocess_fn.udf_filter_text('name', 'text')
)

## Explode the selected arrays in a string, separated by "," 

In [18]:
df_filtered = preprocess_fn.explode_to_strs(df_filtered,
                                            ["colorIdentity", "types", "subtypes", "supertypes"]
)

## Encode newly created strings

In [19]:
from pyspark.ml.feature import StringIndexer, IndexToString

In [20]:
# # df_filtered = preprocess_fn.encode_strings(
#     # df_filtered,
#     # ["rarity", "str_colorIdentity", "str_types", "str_subtypes", "str_supertypes"],
#     # fname=code
# # )
# df = df_filtered

# cols = ["str_colorIdentity"] #, "str_types", "str_subtypes", "str_supertypes"]

# for col in cols:
#     df = df.drop(f"encoded_{col}")

#     indexer = StringIndexer(
#         inputCol=f"{col}", outputCol=f"encoded_{col}", stringOrderType="alphabetAsc"
#     )
#     model = indexer.fit(df)
#     df = model.transform(df)

#     # We'll write the models in a TEMP dictorory and later we'll move them
#     # into our project's subdirectory.
#     # indexer.write().overwrite().save(f"{config.TEMP}/{fname}_stringindexer_{col}")
#     # model.write().overwrite().save(
#     #     f"{config.TEMP}/{fname}_stringindexer_model_{col}"
#     # )

#     # shutil.move(
#     #     f"{config.TEMP}/{fname}_stringindexer_{col}",
#     #     f"{config.SPARK_MODELS}/{fname}/stringindexer_{col}",
#     # )
#     # shutil.move(
#     #     f"{config.TEMP}/{fname}_stringindexer_model_{col}",
#     #     f"{config.SPARK_MODELS}/{fname}/stringindexer_model_{col}",
#     # )

#     indexer.save(f"{config.SPARK_MODELS}/stringindexer_{col}")
#     model.save(f"{config.SPARK_MODELS}/stringindexer_model_{col}")



In [21]:
# mv /tmp/pyspark/stringindexer* /home/makism/Github/mtggg/project/artifacts/spark_models/

## Count the number of colors

In [22]:
df_filtered = df_filtered.withColumn("num_colors", fn.size("colors"))

#  Create an SQL table

In [23]:
df_filtered.createOrReplaceTempView("cards")

In [24]:
tbl = spark.sql("""
    SELECT
        CAST(number as Integer),
        scryfallId,
        name,
        rarity,
        CAST(convertedManaCost as Integer),
        CAST(num_colors as Integer) as numColors,
        str_colorIdentity as colorIdentity,
        -- CAST(encoded_str_colorIdentity as Integer) as encodedColorIdentity,
        str_types as types,
        -- CAST(encoded_str_types as Integer) as encodedTypes,
        str_subtypes as subTypes,
        -- CAST(encoded_str_subtypes as Integer) as encodedSubTypes,
        str_supertypes as superTypes,
        -- CAST(encoded_str_supertypes as Integer) as encodedSuperTypes,
        text as originalText,
        filtered_text as filteredText,
        CAST(power as Integer),
        CAST(toughness as Integer)
    FROM
        cards
""")

# Save to Parquet

In [25]:
tbl.write.mode("overwrite").parquet(f"{config.ARTIFACTS}/datasets/M20_cards.parquet")

In [26]:
## this fails
# tbl.write.mode("overwrite").parquet('dataset/THB_cards.parquet')

In [27]:
# cp -R "/tmp/M20_cards.parquet" "../artifacts/dataset/"